In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import yaml
import os
import urlparse
import requests
import json
import schema_salad.ref_resolver
from wes_service.util import visit

from synorchestrator.trs_client.client import TRSClient
from synorchestrator.wes_client.client import WESClient

In [45]:
with open('../toolregistries.config', 'rb') as f:
    trs_config = yaml.load(f)
    
with open('../workflowservices.config', 'rb') as f:
    wes_config = yaml.load(f)

In [46]:
tclient = TRSClient(**trs_config['dockstore'])
wclient = WESClient(**wes_config['arvados-wes'])

In [47]:
checker_workflow = tclient.get_workflow_checker('github.com/dockstore-testing/md5sum-checker')
checker_workflow['id']

DEBUG:root:retrieving workflow entry from tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dockstore.org
DEBUG:urllib3.connectionpool:https://dockstore.org:8443 "GET /api/ga4gh/v2/tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker HTTP/1.1" 200 418
DEBUG:synorchestrator.trs_client.client:found checker workflow: github.com/dockstore-testing/md5sum-checker/_cwl_checker
DEBUG:root:retrieving workflow entry from tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker%2F_cwl_checker
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dockstore.org
DEBUG:urllib3.connectionpool:https://dockstore.org:8443 "GET /api/ga4gh/v2/tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker%2F_cwl_checker HTTP/1.1" 200 496


u'#workflow/github.com/dockstore-testing/md5sum-checker/_cwl_checker'

In [50]:
checker_descriptor = tclient.get_workflow_descriptor(checker_workflow['id'], 'develop', 'CWL')
checker_descriptor['url']

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dockstore.org
DEBUG:urllib3.connectionpool:https://dockstore.org:8443 "GET /api/ga4gh/v2/tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker%2F_cwl_checker/versions/develop/CWL/descriptor HTTP/1.1" 200 541


u'https://raw.githubusercontent.com/dockstore-testing/md5sum-checker/develop/checker-workflow-wrapping-workflow.cwl'

In [52]:
checker_tests = tclient.get_workflow_tests(checker_workflow['id'], 'develop', 'CWL')
checker_tests[0]

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dockstore.org
DEBUG:urllib3.connectionpool:https://dockstore.org:8443 "GET /api/ga4gh/v2/tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker%2F_cwl_checker/versions/develop/CWL/tests HTTP/1.1" 200 206


{u'test': u'{\n  "input_file": {\n        "class": "File",\n        "path": "md5sum.input"\n    },\n    "expected_md5": "00579a00e3e7fa0674428ac7049423e2"\n}\n',
 u'url': u'/checker-input-cwl.json'}

In [53]:
checker_test_url = os.path.join(
    os.path.dirname(checker_descriptor['url']), 
    os.path.basename(checker_tests[0]['url'])
)
checker_test_url

u'https://raw.githubusercontent.com/dockstore-testing/md5sum-checker/develop/checker-input-cwl.json'

In [15]:
data = {
    "workflow_url": checker_descriptor['url'],
    "workflow_params": checker_test_url,
    "workflow_type": "CWL",
    "workflow_type_version": "v1.0"
}
data

{'workflow_params': u'https://raw.githubusercontent.com/dockstore-testing/md5sum-checker/develop/checker-input-cwl.json',
 'workflow_type': 'CWL',
 'workflow_type_version': 'v1.0',
 'workflow_url': u'https://raw.githubusercontent.com/dockstore-testing/md5sum-checker/develop/checker-workflow-wrapping-workflow.cwl'}

In [34]:
wclient.get_service_info()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): wes.qr1hi.arvadosapi.com
DEBUG:urllib3.connectionpool:https://wes.qr1hi.arvadosapi.com:443 "GET /ga4gh/wes/v1/service-info HTTP/1.1" 200 325


{u'engine_versions': u'cwl-runner',
 u'key_values': {},
 u'supported_filesystem_protocols': [u'file', u'http', u'https', u'keep'],
 u'supported_wes_versions': u'0.2.1',
 u'system_state_counts': {},
 u'workflow_type_versions': {u'CWL': {u'workflow_type_version': [u'v1.0']}}}

In [36]:
wclient.list_workflow_runs()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): wes.qr1hi.arvadosapi.com
DEBUG:urllib3.connectionpool:https://wes.qr1hi.arvadosapi.com:443 "GET /ga4gh/wes/v1/workflows HTTP/1.1" 200 2231


{u'next_page_token': u'',
 u'workflows': [{u'state': u'COMPLETE',
   u'workflow_id': u'qr1hi-xvhdp-3s6qt9v3p53agsu'},
  {u'state': u'COMPLETE', u'workflow_id': u'qr1hi-xvhdp-57pef89pozypl2z'},
  {u'state': u'COMPLETE', u'workflow_id': u'qr1hi-xvhdp-6qud0jj07rdnvx6'},
  {u'state': u'COMPLETE', u'workflow_id': u'qr1hi-xvhdp-8zn3q04rkdg4x52'},
  {u'state': u'COMPLETE', u'workflow_id': u'qr1hi-xvhdp-996c4y2xynso9qh'},
  {u'state': u'COMPLETE', u'workflow_id': u'qr1hi-xvhdp-9thf0yjwnjh6h73'},
  {u'state': u'COMPLETE', u'workflow_id': u'qr1hi-xvhdp-9vch6kqxifwbv45'},
  {u'state': u'COMPLETE', u'workflow_id': u'qr1hi-xvhdp-ao0meq3szjq6bwz'},
  {u'state': u'QUEUED', u'workflow_id': u'qr1hi-xvhdp-d9fubcd90k25fh3'},
  {u'state': u'COMPLETE', u'workflow_id': u'qr1hi-xvhdp-ghv7l7mmvs6mifw'},
  {u'state': u'COMPLETE', u'workflow_id': u'qr1hi-xvhdp-h04nggwn6vzt5oz'},
  {u'state': u'COMPLETE', u'workflow_id': u'qr1hi-xvhdp-hwk8t8j8k7mgskw'},
  {u'state': u'COMPLETE', u'workflow_id': u'qr1hi-xvhdp-i1x

In [38]:
wclient.get_workflow_run('qr1hi-xvhdp-ugwoi4ipmvhsoir')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): wes.qr1hi.arvadosapi.com
DEBUG:urllib3.connectionpool:https://wes.qr1hi.arvadosapi.com:443 "GET /ga4gh/wes/v1/workflows/qr1hi-xvhdp-ugwoi4ipmvhsoir HTTP/1.1" 200 2384


{u'outputs': {u'workflow_output_file': {u'basename': u'results.json',
   u'class': u'File',
   u'location': u'https://download.qr1hi.arvadosapi.com/c=51a1c8291787268318b8e84da2371019+117/_/results.json',
   u'size': 57}},
 u'request': {},
 u'state': u'COMPLETE',
 u'task_logs': [],
 u'workflow_id': u'qr1hi-xvhdp-ugwoi4ipmvhsoir',
 u'workflow_log': {u'cmd': [u''],
  u'endTime': u'',
  u'exit_code': 0,
  u'startTime': u'',
  u'stderr': u'2018-05-15T15:20:10.578578670Z cwltool INFO: /usr/local/bin/arvados-cwl-runner 1.1.4.20180511203654, arvados-python-client 1.1.4.20180510153813, cwltool 1.0.20180508202931\n2018-05-15T15:20:10.580624270Z cwltool INFO: Resolved \'/var/lib/cwl/workflow.json#main\' to \'file:///var/lib/cwl/workflow.json#main\'\n2018-05-15T15:20:12.682419170Z cwltool INFO: [workflow workflow.json#main] start\n2018-05-15T15:20:12.683911770Z cwltool INFO: [step md5sum] start\n2018-05-15T15:20:12.686741870Z cwltool INFO: [workflow md5sum] start\n2018-05-15T15:20:12.688149770Z cw

In [18]:
loader = schema_salad.ref_resolver.Loader({
    "location": {"@type": "@id"},
    "path": {"@type": "@id"}
})
input, _ = loader.resolve_ref(checker_test_url)

DEBUG:cachecontrol.controller:Looking up "https://raw.githubusercontent.com/dockstore-testing/md5sum-checker/develop/checker-input-cwl.json" in the cache
DEBUG:cachecontrol.controller:No cache entry available
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "GET /dockstore-testing/md5sum-checker/develop/checker-input-cwl.json HTTP/1.1" 200 125
DEBUG:cachecontrol.controller:Updating cache with response from "https://raw.githubusercontent.com/dockstore-testing/md5sum-checker/develop/checker-input-cwl.json"
DEBUG:cachecontrol.controller:Caching due to etag
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "HEAD /dockstore-testing/md5sum-checker/develop/md5sum.input HTTP/1.1" 200 0


In [40]:
input

ordereddict([('input_file', ordereddict([('class', 'File'), ('location', u'https://raw.githubusercontent.com/dockstore-testing/md5sum-checker/develop/md5sum.input')])), ('expected_md5', '00579a00e3e7fa0674428ac7049423e2')])

In [22]:
basedir = os.path.dirname(checker_test_url)
def fixpaths(d):
    if isinstance(d, dict):
        if "path" in d:
            if ":" not in d["path"]:
                local_path = os.path.normpath(
                    os.path.join(os.getcwd(), basedir, d["path"]))
                d["location"] = urllib.pathname2url(local_path)
            else:
                d["location"] = d["path"]
            del d["path"]
        if d.get("class") == "Directory":
            loc = d.get("location", "")
            if loc.startswith("http:") or loc.startswith("https:"):
                logging.error("Directory inputs not supported with http references")
                exit(33)

visit(input, fixpaths)

In [24]:
data = {
    "workflow_url": checker_descriptor['url'],
    "workflow_params": input,
    "workflow_type": "CWL",
    "workflow_type_version": "v1.0"
}
data

{'workflow_params': ordereddict([('input_file', ordereddict([('class', 'File'), ('location', u'https://raw.githubusercontent.com/dockstore-testing/md5sum-checker/develop/md5sum.input')])), ('expected_md5', '00579a00e3e7fa0674428ac7049423e2')]),
 'workflow_type': 'CWL',
 'workflow_type_version': 'v1.0',
 'workflow_url': u'https://raw.githubusercontent.com/dockstore-testing/md5sum-checker/develop/checker-workflow-wrapping-workflow.cwl'}